# SPKW Redundancy Analysis

This notebook analyzes redundancy between Swiss-Prot Keyword (SPKW) annotations and experimental evidence.

**Pre-requisite**: Run `just export-annotations-duckdb` to generate the DuckDB database with redundancy relationships.

In [1]:
import duckdb
import pandas as pd
import os

# Change to project root if running from projects directory
if os.path.basename(os.getcwd()) == 'projects':
    os.chdir('..')

conn = duckdb.connect('exports/annotations.duckdb', read_only=True)

# Quick overview
print("=== Database Overview ===")
print(f"Total annotations: {conn.execute('SELECT COUNT(*) FROM annotations').fetchone()[0]:,}")
print(f"Total redundancy relationships: {conn.execute('SELECT COUNT(*) FROM redundant_with').fetchone()[0]:,}")

=== Database Overview ===
Total annotations: 42,053
Total redundancy relationships: 190,236


## Annotation Counts by Evidence Type and Species

In [2]:
# Annotations by species and whether they are SPKW-derived
species_spkw = conn.execute("""
    SELECT 
        taxon_label,
        CASE WHEN original_reference_id = 'GO_REF:0000043' THEN 'SPKW' ELSE 'Other' END as source,
        COUNT(*) as count
    FROM annotations
    GROUP BY taxon_label, source
    ORDER BY taxon_label, source
""").fetchdf()

# Pivot to show side by side
species_pivot = species_spkw.pivot(index='taxon_label', columns='source', values='count').fillna(0).astype(int)
species_pivot['Total'] = species_pivot.sum(axis=1)
species_pivot['SPKW %'] = (species_pivot['SPKW'] / species_pivot['Total'] * 100).round(1)
species_pivot = species_pivot.sort_values('Total', ascending=False)
print("=== Annotations by Species and Source ===")
display(species_pivot)

=== Annotations by Species and Source ===


source,Other,SPKW,Total,SPKW %
taxon_label,,,,
Homo sapiens,25724,825,26549,3.1
Mus musculus,5839,109,5948,1.8
Caenorhabditis elegans,2670,236,2906,8.1
Saccharomyces cerevisiae,1706,133,1839,7.2
Arabidopsis thaliana,1157,65,1222,5.3
...,...,...,...,...
Caenorhabditis briggsae,4,1,5,20.0
Pseudomonas entomophila L48,4,0,4,0.0
Phocaeicola vulgatus CL09T03C04,3,0,3,0.0


## SPKW Redundancy Overview

How many SPKW annotations are made redundant by more specific experimental evidence?

In [3]:
# SPKW annotations that have experimental evidence making them redundant
redundancy_summary = conn.execute("""
    WITH spkw_annotations AS (
        SELECT id, gene_symbol, term_id, term_label, taxon_label
        FROM annotations
        WHERE original_reference_id = 'GO_REF:0000043'
    ),
    spkw_with_redundancy AS (
        SELECT DISTINCT s.id, s.gene_symbol, s.term_label, s.taxon_label, r.relationship
        FROM spkw_annotations s
        JOIN redundant_with r ON s.id = r.general_annotation_id
        JOIN annotations a2 ON r.specific_annotation_id = a2.id
        WHERE a2.original_reference_id != 'GO_REF:0000043'
    )
    SELECT 
        taxon_label,
        relationship,
        COUNT(DISTINCT id) as redundant_spkw_annotations,
        COUNT(DISTINCT gene_symbol) as genes_affected
    FROM spkw_with_redundancy
    GROUP BY taxon_label, relationship
    ORDER BY taxon_label, relationship
""").fetchdf()

print("=== SPKW Annotations Subsumed by Experimental Evidence ===")
display(redundancy_summary)

=== SPKW Annotations Subsumed by Experimental Evidence ===


,taxon_label,relationship,redundant_spkw_annotations,genes_affected
0,Acetivibrio thermocellus,ISA_PARTOF,13,6
1,Acetivibrio thermocellus (strain ATCC 27405 / ...,ISA_PARTOF,14,7
2,Acetobacter pasteurianus,ISA_PARTOF,1,1
3,Agkistrodon contortrix contortrix,ISA_PARTOF,2,1
4,Anopheles gambiae,EXACT,3,3
...,...,...,...,...
83,Starmerella bombicola,ISA_PARTOF,1,1
84,Streptomyces coelicolor A3(2),ISA_PARTOF,1,1
85,Tequatrovirus,ISA_PARTOF,2,1
86,Trichoderma reesei,ISA_PARTOF,3,1


In [4]:
# Pivot for cleaner view
if not redundancy_summary.empty:
    redundancy_pivot = redundancy_summary.pivot(
        index='taxon_label', 
        columns='relationship', 
        values='redundant_spkw_annotations'
    ).fillna(0).astype(int)
    redundancy_pivot['Total Redundant'] = redundancy_pivot.sum(axis=1)
    redundancy_pivot = redundancy_pivot.sort_values('Total Redundant', ascending=False)
    print("=== SPKW Redundancy by Species and Relationship Type ===")
    display(redundancy_pivot)

=== SPKW Redundancy by Species and Relationship Type ===


relationship,EXACT,ISA_PARTOF,REGULATES,Total Redundant
taxon_label,,,,
Homo sapiens,100,454,158,712
Caenorhabditis elegans,16,164,25,205
Saccharomyces cerevisiae,16,87,25,128
Mus musculus,10,69,25,104
Bacillus subtilis (strain 168),5,41,11,57
...,...,...,...,...
Bacillus subtilis subsp. subtilis str. 168,0,1,0,1
Methanothrix thermoacetophila PT,0,1,0,1
Methanosarcina acetivorans C2A,0,1,0,1


## Top Redundant SPKW Terms

Which SPKW-derived GO terms are most often made redundant by experimental evidence?

In [5]:
top_redundant_terms = conn.execute("""
    SELECT 
        r.general_term_label as spkw_term,
        r.relationship,
        COUNT(DISTINCT r.gene_symbol) as genes,
        COUNT(*) as annotation_pairs
    FROM redundant_with r
    JOIN annotations a1 ON r.general_annotation_id = a1.id
    JOIN annotations a2 ON r.specific_annotation_id = a2.id
    WHERE a1.original_reference_id = 'GO_REF:0000043'
      AND a2.original_reference_id != 'GO_REF:0000043'
    GROUP BY r.general_term_label, r.relationship
    ORDER BY genes DESC
    LIMIT 30
""").fetchdf()

print("=== Top 30 SPKW Terms Subsumed by Experimental Evidence ===")
display(top_redundant_terms)

=== Top 30 SPKW Terms Subsumed by Experimental Evidence ===


,spkw_term,relationship,genes,annotation_pairs
0,hydrolase activity,ISA_PARTOF,110,545
1,transferase activity,ISA_PARTOF,104,2712
2,metal ion binding,ISA_PARTOF,91,267
3,nucleotide binding,ISA_PARTOF,86,192
4,DNA-templated transcription,REGULATES,50,697
5,DNA binding,ISA_PARTOF,44,287
6,apoptotic process,REGULATES,37,419
7,oxidoreductase activity,ISA_PARTOF,37,157
8,kinase activity,ISA_PARTOF,35,629
9,immune system process,ISA_PARTOF,35,243


## Regulatory Conflation Analysis

Cases where SPKW says gene is "involved in X" but experimental evidence shows it "regulates X".
This is a key pattern indicating SPKW over-annotation.

In [6]:
regulatory_conflation = conn.execute("""
    SELECT 
        r.general_term_label as spkw_term,
        COUNT(DISTINCT r.gene_symbol) as genes,
        COUNT(*) as cases
    FROM redundant_with r
    JOIN annotations a1 ON r.general_annotation_id = a1.id
    JOIN annotations a2 ON r.specific_annotation_id = a2.id
    WHERE a1.original_reference_id = 'GO_REF:0000043'
      AND a2.original_reference_id != 'GO_REF:0000043'
      AND r.relationship = 'REGULATES'
    GROUP BY r.general_term_label
    ORDER BY genes DESC
    LIMIT 20
""").fetchdf()

print("=== SPKW Terms with Regulatory Conflation (SPKW says 'X', Experimental says 'regulates X') ===")
display(regulatory_conflation)

=== SPKW Terms with Regulatory Conflation (SPKW says 'X', Experimental says 'regulates X') ===


,spkw_term,genes,cases
0,DNA-templated transcription,50,697
1,apoptotic process,37,419
2,autophagy,19,57
3,immune system process,17,80
4,cell differentiation,13,60
5,rhythmic process,11,38
6,innate immune response,9,22
7,cell division,8,13
8,transferase activity,7,14
9,lipid metabolic process,7,11


In [7]:
# Examples of regulatory conflation
regulatory_examples = conn.execute("""
    SELECT 
        r.gene_symbol,
        r.general_term_label as spkw_says,
        r.specific_term_label as experimental_says,
        a2.evidence_type
    FROM redundant_with r
    JOIN annotations a1 ON r.general_annotation_id = a1.id
    JOIN annotations a2 ON r.specific_annotation_id = a2.id
    WHERE a1.original_reference_id = 'GO_REF:0000043'
      AND a2.original_reference_id != 'GO_REF:0000043'
      AND r.relationship = 'REGULATES'
    ORDER BY r.gene_symbol
    LIMIT 30
""").fetchdf()

print("=== Sample Regulatory Conflation Cases ===")
display(regulatory_examples)

=== Sample Regulatory Conflation Cases ===


,gene_symbol,spkw_says,experimental_says,evidence_type
0,ABL1,autophagy,regulation of autophagy,IEA
1,ABL1,DNA repair,negative regulation of double-strand break rep...,IDA
2,ABL1,endocytosis,regulation of endocytosis,IEA
3,ABL1,transferase activity,negative regulation of ubiquitin-protein trans...,IDA
4,ABL1,cell adhesion,positive regulation of substrate adhesion-depe...,IMP
5,ABL1,DNA damage response,negative regulation of double-strand break rep...,IDA
6,ABL1,autophagy,regulation of autophagy,TAS
7,ABL1,transferase activity,protein serine/threonine kinase activator acti...,IDA
8,ABL1,cell adhesion,positive regulation of establishment of T cell...,ISS
9,ABL1,cell adhesion,positive regulation of establishment of T cell...,IEA


## Swiss-Prot vs TrEMBL Breakdown

In [8]:
# Check if is_swissprot column exists
columns = conn.execute("DESCRIBE annotations").fetchdf()
has_swissprot = 'is_swissprot' in columns['column_name'].values

if has_swissprot:
    swissprot_breakdown = conn.execute("""
        SELECT 
            CASE WHEN is_swissprot THEN 'Swiss-Prot' ELSE 'TrEMBL' END as db,
            CASE WHEN original_reference_id = 'GO_REF:0000043' THEN 'SPKW' ELSE 'Other' END as source,
            COUNT(*) as annotations,
            COUNT(DISTINCT gene_symbol) as genes
        FROM annotations
        WHERE is_swissprot IS NOT NULL
        GROUP BY db, source
        ORDER BY db, source
    """).fetchdf()
    print("=== Annotations by Swiss-Prot/TrEMBL Status ===")
    display(swissprot_breakdown)
else:
    print("Note: is_swissprot column not present. Re-run export-annotations-duckdb to include it.")

=== Annotations by Swiss-Prot/TrEMBL Status ===


,db,source,annotations,genes
0,Swiss-Prot,Other,38193,732
1,Swiss-Prot,SPKW,1703,610
2,TrEMBL,Other,939,113
3,TrEMBL,SPKW,175,74


## Review Action Distribution

In [9]:
# Actions by species and SPKW status
action_distribution = conn.execute("""
    SELECT 
        taxon_label,
        CASE WHEN original_reference_id = 'GO_REF:0000043' THEN 'SPKW' ELSE 'Other' END as source,
        "review.action" as review_action,
        COUNT(*) as count
    FROM annotations
    WHERE "review.action" IS NOT NULL
    GROUP BY taxon_label, source, "review.action"
    ORDER BY taxon_label, source, count DESC
""").fetchdf()

print("=== Review Actions by Species and Source ===")
display(action_distribution)

=== Review Actions by Species and Source ===


,taxon_label,source,review_action,count
0,Acetivibrio thermocellus,Other,NEW,19
1,Acetivibrio thermocellus,Other,ACCEPT,18
2,Acetivibrio thermocellus,Other,MODIFY,9
3,Acetivibrio thermocellus,Other,REMOVE,6
4,Acetivibrio thermocellus,Other,KEEP_AS_NON_CORE,4
...,...,...,...,...
505,Xenopus tropicalis,Other,NEW,11
506,Xenopus tropicalis,Other,REMOVE,2
507,Xenopus tropicalis,Other,MODIFY,1
508,Xenopus tropicalis,Other,ACCEPT,1


In [10]:
# Summary: SPKW over-annotation rates
overannotation_rates = conn.execute("""
    SELECT 
        taxon_label,
        COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043') as spkw_total,
        COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                         AND "review.action" = 'MARK_AS_OVER_ANNOTATED') as spkw_overannotated,
        ROUND(100.0 * COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                                        AND "review.action" = 'MARK_AS_OVER_ANNOTATED') /
              NULLIF(COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043'), 0), 1) as overannotation_rate_pct
    FROM annotations
    GROUP BY taxon_label
    HAVING COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043') > 0
    ORDER BY overannotation_rate_pct DESC NULLS LAST
""").fetchdf()

print("=== SPKW Over-annotation Rates by Species ===")
display(overannotation_rates)

=== SPKW Over-annotation Rates by Species ===


,taxon_label,spkw_total,spkw_overannotated,overannotation_rate_pct
0,Gallus gallus,1,1,100.0
1,Azotobacter vinelandii,3,2,66.7
2,Nitratidesulfovibrio vulgaris (strain Hildenbo...,4,2,50.0
3,Clostridium cellulovorans,9,4,44.4
4,Schizosaccharomyces pombe,5,2,40.0
...,...,...,...,...
66,Agkistrodon contortrix contortrix,4,0,0.0
67,Caenorhabditis briggsae,1,0,0.0
68,Penicillium chrysogenum,4,0,0.0
69,Pseudomonas putida KT2440,5,0,0.0


In [11]:
# Over-annotation rates by Swiss-Prot status
if has_swissprot:
    sp_overannotation = conn.execute("""
        SELECT 
            CASE WHEN is_swissprot IS TRUE THEN 'Swiss-Prot' 
                 WHEN is_swissprot IS FALSE THEN 'TrEMBL' 
                 ELSE 'Unknown' END as db,
            COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043') as spkw_total,
            COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                             AND "review.action" = 'MARK_AS_OVER_ANNOTATED') as spkw_overannotated,
            COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                             AND "review.action" = 'ACCEPT') as spkw_accepted,
            COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                             AND "review.action" = 'MODIFY') as spkw_modified,
            ROUND(100.0 * COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043' 
                                            AND "review.action" = 'MARK_AS_OVER_ANNOTATED') /
                  NULLIF(COUNT(*) FILTER (WHERE original_reference_id = 'GO_REF:0000043'), 0), 1) as overannotation_pct
        FROM annotations
        GROUP BY db
        ORDER BY db
    """).fetchdf()
    print("=== SPKW Over-annotation Rates by Swiss-Prot Status ===")
    display(sp_overannotation)

=== SPKW Over-annotation Rates by Swiss-Prot Status ===


,db,spkw_total,spkw_overannotated,spkw_accepted,spkw_modified,overannotation_pct
0,Swiss-Prot,1703,182,786,188,10.7
1,TrEMBL,175,19,60,32,10.9
2,Unknown,116,13,60,6,11.2


In [12]:
conn.close()